<div class="jumbotron">
  <h1><i class="fa fa-bar-chart" aria-hidden="true"></i> Análisis de Datos</h1>
  <p></p>
</div>

In [89]:
import csv
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from scipy import stats
import matplotlib.mlab as mlab
import datetime as dt
#import plotly.plotly as py  # tools to communicate with Plotly's server

## Cuestiones

- Tiempo estimado de la duración de una llamada externa a recepción para  ser redirigida
- Valor promedio del numero de llamadas por día 
- Valor promedio de días habiles por año
- Numero de llamadas  de  externas (PSTN) los cuales llarcan directamente la extesión interna (DTMF)
- Numero de llamadas  de  internas entre extesiones 
 




## Leer los datos 

In [4]:
data = pd.read_csv('CDRReport-2018Feb16.094746_v2.csv')
data.head(10)

,Date,Source,Destination,Status,Duration
0,31/01/2018 18:08,8.182630e+09,2,ANSWERED,23s
1,31/01/2018 18:06,8.182630e+09,2,ANSWERED,13s
2,31/01/2018 18:05,8.182630e+09,2,ANSWERED,13s
3,31/01/2018 18:00,8.182630e+09,2,ANSWERED,10s
4,31/01/2018 17:53,5.555113e+09,2,ANSWERED,13s
5,31/01/2018 17:35,5.550806e+09,2,ANSWERED,12s
6,31/01/2018 17:26,5.559722e+09,2,ANSWERED,35s
7,31/01/2018 17:24,5.553872e+09,2,ANSWERED,12s
8,31/01/2018 17:24,5.519497e+09,2,ANSWERED,9s
9,31/01/2018 17:16,4.881011e+09,2,ANSWERED,15s


## Número de muestras

In [5]:
print("Número de registros :", len(data['Date']))

Número de registros : 20480


## Primer paso es buscar valores nulos

In [6]:
print("El número de valores nulos en el dataframe es : \n" , data.isnull().sum())

El número de valores nulos en el dataframe es : 
 Date           0
Source         7
Destination    0
Status         0
Duration       0
dtype: int64


## Eliminar valores nulos

In [7]:
data = data.dropna()

In [9]:
print("Número de muestras después de eliminar los datos tipo NaN: ",len(data))

Número de muestras después de eliminar los datos tipo NaN:  20473


## Carmbiar tipo de dato de la variable Source

In [10]:
data.Source = data.Source.astype('int64')

In [11]:
data.head(2)

,Date,Source,Destination,Status,Duration
0,31/01/2018 18:08,8182629900,2,ANSWERED,23s
1,31/01/2018 18:06,8182629900,2,ANSWERED,13s


## Formato a la variable Duration

In [9]:
#data.Duration.replace('s','')

In [13]:
def split_s(x):
    return x.split('s')[0]

In [14]:
data['Duration'] = data['Duration'].apply(split_s)

In [15]:
data['Duration'] = data['Duration'].apply(lambda x: x.split('s')[0])

In [16]:
data['Duration'] = data['Duration'].astype(int)

## Formato a las variable Date

In [17]:
data.Date = pd.to_datetime(data.Date,infer_datetime_format=True)
#pd.to_datetime(data.Date, format='%Y%m%d %H%M') #errors='ignore'

In [18]:
data.head()

,Date,Source,Destination,Status,Duration
0,2018-01-31 18:08:00,8182629900,2,ANSWERED,23
1,2018-01-31 18:06:00,8182629900,2,ANSWERED,13
2,2018-01-31 18:05:00,8182629900,2,ANSWERED,13
3,2018-01-31 18:00:00,8182629900,2,ANSWERED,10
4,2018-01-31 17:53:00,5555113237,2,ANSWERED,13


In [19]:
print("Total de muestras provenientes  de llamadas externas : " ,len(data[data['Source'].astype('str').str.len() > 9]))

Total de muestras provenientes  de llamadas externas :  18655


In [20]:
print("Total de muestras provenientes  de llamadas internas : " ,len(data[data['Source'].astype('str').str.len() <= 9]))

Total de muestras provenientes  de llamadas internas :  1818


In [21]:
print("Llamadas externas + llamadas internas : " ,len(data[data['Source'].astype('str').str.len() > 9])+len(data[data['Source'].astype('str').str.len() <= 9]),"  --->consistencia")

Llamadas externas + llamadas internas :  20473   --->consistencia


## Llamadas  respondidas 

In [22]:
data = data[(data.Status == 'ANSWERED')&(data['Source'].astype('str').str.len() > 9)]

In [23]:
data.shape

(16605, 5)

In [24]:
data.tail(5)

,Date,Source,Destination,Status,Duration
20473,2017-02-01 09:46:00,8183574404,2,ANSWERED,10
20474,2017-02-01 09:44:00,5550004400,2,ANSWERED,11
20475,2017-02-01 09:24:00,5568402801,2,ANSWERED,13
20477,2017-02-01 09:13:00,5536841610,2,ANSWERED,16
20478,2017-02-01 09:02:00,5553082744,2,ANSWERED,32


In [25]:
print("Duración del total de llamadas",data.Duration.sum(),"segs")
print("Duración del total de llamadas", data.Duration.sum()/60,"mins")
print("Duración del total de llamadas", data.Duration.sum()/3600,"hrs" )

Duración del total de llamadas 443425 segs
Duración del total de llamadas 7390.416666666667 mins
Duración del total de llamadas 123.17361111111111 hrs


- Ya sabesmos que todas son a recepción y que fuerón respondidas


In [50]:
del data['Status']
del data ['Destination']

In [51]:
data

,Date,Source,Duration
0,2018-01-31 18:08:00,8182629900,23
1,2018-01-31 18:06:00,8182629900,13
2,2018-01-31 18:05:00,8182629900,13
3,2018-01-31 18:00:00,8182629900,10
4,2018-01-31 17:53:00,5555113237,13
...,...,...,...
20473,2017-02-01 09:46:00,8183574404,10
20474,2017-02-01 09:44:00,5550004400,11
20475,2017-02-01 09:24:00,5568402801,13
20477,2017-02-01 09:13:00,5536841610,16


In [59]:
#pd.DataFrame(data.groupby(pd.Grouper(key='Date', freq='1M')))
#T.groupby(pd.Grouper(key='date', freq='1M')).sum()
#T.groupby(T['date'].dt.strftime('%B'))['revenue'].sum()

In [63]:
 gb.groups.keys()

dict_keys([(2017, 2), (2017, 3), (2017, 4), (2017, 5), (2017, 6), (2017, 7), (2017, 8), (2017, 9), (2017, 10), (2017, 11), (2017, 12), (2018, 1)])

In [117]:
#gb = data.groupby([data['Date'].dt.year])['Date','Source','Duration']
#gb = data.groupby([data['Date'].dt.year,data['Date'].dt.month])['Date','Source','Duration']
#for key_year in gb.groups.keys():
#    df_year = gb.get_group(key_year)
    #print(df_year)
#.rename_axis('Año').reset_index()
#,data['Date'].dt.month,data['Date'].dt.day]

In [69]:
df_gb_4 = pd.DataFrame(data.groupby([data['Date'].dt.year,data['Date'].dt.month,data['Date'].dt.day]).Source.count())

In [110]:
df_gb_4[df_gb_4.Source < 27 ].Source.sum()

51

In [108]:
df_gb_4[df_gb_4.Source < 27 ].shape

(10, 1)

In [86]:
date_ = list(df_gb_4[df_gb_4.Source < 27 ].index)

In [104]:
len(date_)

10

In [101]:
list_date = []
for a,m,d in date_:
    list_date.append(dt.date(a,m,d))

In [102]:
list_date

[datetime.date(2017, 2, 6),
 datetime.date(2017, 3, 20),
 datetime.date(2017, 4, 13),
 datetime.date(2017, 4, 16),
 datetime.date(2017, 8, 12),
 datetime.date(2017, 8, 13),
 datetime.date(2017, 11, 2),
 datetime.date(2017, 11, 20),
 datetime.date(2017, 12, 25),
 datetime.date(2018, 1, 1)]

In [111]:
data[ data.Date.dt.date.isin(list_date)].shape

(51, 3)

## Agg

In [133]:
gb = data.groupby([data['Date'].dt.year,data['Date'].dt.month,data['Date'].dt.day]).agg(
    {
         'Duration':sum,    # Sum duration per group
         'Duration': "median",  # get the count of 

    }
)

In [134]:
df_  = pd.DataFrame(gb)

In [135]:
df_

Duration
Date Date Date          
2017 2    1         13.0
          2         18.0
          3         18.0
          6         26.0
          7         16.0
...                  ...
2018 1    25        13.5
          26        17.0
          29        20.0
          30        21.0
          31        15.0

[262 rows x 1 columns]

## Eliminar los datos atípicos 

- Días con pocas lladas
- Días festivos 
- Sabados y domingos 

- Llamadas fuera de horario 

In [ ]:
#for key_year in gd_year.groups.keys():
    #df_year = gd_year.get_group(key_year)
    #gb_month = df_year.groupby(data['Date'].dt.month)